<a href="https://colab.research.google.com/github/etmcrae/DDDS-Cohort-16-Projects/blob/main/Housing_Description_for_Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Housing Price Prediction


The project should be done using Regression prediction techniques ( with feature scaling and regularization ) as covered in class.
The goal is to minimize the RMS ***percentage*** error ( root mean squared percentage error - RMSPE ) on your prediction of the house sales price.



Any of the available features can be used in the regression, but a minimum set of variables that do the job should be the ultimate goal.



This project will have less direct "To Do" guidance and the progression through the data science process will be more individualized this time around.
We will talk about all the issues during class so you're not going to be out on a ledge with this one, I just want to move you towards performing a data science project on your own eventually.



## Problem Defition

Fill this in at the end

## Data Collection

These initial first steps import libraries and the data source for analysis.

In [219]:
#import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [220]:
#this serves as an unmodified copy of the dataframe as a backup
url_UNMODIFIED = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv'
df_UNMODIFIED = pd.read_csv(url_UNMODIFIED)

In [221]:
#this is the dataframe imported from the url
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv'
df = pd.read_csv(url)

In [222]:
#this prints the first 5 rows of the dataframe
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,905101070,20,RL,62.0,14299,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2007,WD,Normal,115400
1,905101330,90,RL,72.0,10791,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2006,WD,Normal,90000
2,903454090,50,RM,50.0,9000,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,12,2007,WD,Normal,141000
3,533244030,60,FV,68.0,7379,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,254000
4,909252020,70,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2009,WD,Normal,155000


In [223]:
#This step performs multiple valuable summaries and pulls key information to gleam
#insights into the data set for an initial look at the overview of the data.
print(df.shape)
print(type(df.shape))
print(type(df.info))
print(df.describe().columns)
print(df.describe())
print(df.info)
print(df.dtypes)


(2637, 81)
<class 'tuple'>
<class 'method'>
Index(['PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area',
       'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area',
       'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold', 'SalePrice'],
      dtype='object')
                PID  MS SubClass  Lot Frontage       Lot Area  Overall Qual  \
count  2.637000e+03  2637.000000   2188.000000    2637.000000   2637.000000   
mean   7.141301e+08    57.349261     69.166819   10044.694729      6.097459   
std    1.887527e+08    42.499091     23.356779    6742.549521 

In [224]:
#This step looks at the occurance of null values throughout the dataset by columns,
#and sorts the columns by highest percent null values.
nulls = df.isnull().sum()
filter = nulls > 0
( nulls[ filter ].sort_values( ascending = False ) / df.shape[0] * 100 ).round(1)


,0
Pool QC,99.6
Misc Feature,96.4
Alley,93.2
Fence,80.0
Mas Vnr Type,60.9
Fireplace Qu,48.7
Lot Frontage,17.0
Garage Cond,5.6
Garage Yr Blt,5.6
Garage Finish,5.6


## Data Cleaning

In [225]:
df.drop(['Alley','Mas Vnr Type','Fireplace Qu','Pool QC','Fence','Misc Feature',], axis = 1, inplace = True)

### You want to fill the null values with either mean or mode. Usually mean is good becasue it helps mantain the normal distribution of your data. If you fill with 0 it will casue issues when training the model

In [226]:
# #This step fills the remaining null values with zeros. Zeros are chosen so they will have minimum impact on the calculation of the values.
# ctrl + ? to put all # to comment all rows out
# df = df.fillna(0)
# nulls = df.isnull().sum()
# filter = nulls > 0
# ( nulls[ filter ].sort_values( ascending = False ) / df.shape[0] * 100 ).round(1)

### This is an example changing the null values with the mode

In [227]:
# object-type columns
#ctrl a then ctrl + ? + down. try once and then twice
object_columns = df.select_dtypes(include=['object'])

for col in object_columns:
    df[col].fillna(df[col].mode()[0] , inplace=True)
object_columns.isnull().sum()

<ipython-input-227-34c09b73236f>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0] , inplace=True)


,0
MS Zoning,0
Street,0
Lot Shape,0
Land Contour,0
Utilities,0
Lot Config,0
Land Slope,0
Neighborhood,0
Condition 1,0
Condition 2,0


In [228]:
object_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MS Zoning       2637 non-null   object
 1   Street          2637 non-null   object
 2   Lot Shape       2637 non-null   object
 3   Land Contour    2637 non-null   object
 4   Utilities       2637 non-null   object
 5   Lot Config      2637 non-null   object
 6   Land Slope      2637 non-null   object
 7   Neighborhood    2637 non-null   object
 8   Condition 1     2637 non-null   object
 9   Condition 2     2637 non-null   object
 10  Bldg Type       2637 non-null   object
 11  House Style     2637 non-null   object
 12  Roof Style      2637 non-null   object
 13  Roof Matl       2637 non-null   object
 14  Exterior 1st    2637 non-null   object
 15  Exterior 2nd    2637 non-null   object
 16  Exter Qual      2637 non-null   object
 17  Exter Cond      2637 non-null   object
 18  Foundati

In [229]:
numerical_columns = df.select_dtypes(include=['int', 'float'])

In [230]:
numerical_columns.shape

(2637, 38)

In [231]:
# # Build a data frame to contain the metadata, using the generic name
# df_info = pd.concat( [
#   pd.DataFrame( [ df.isna().sum().to_dict() ], index = ["Nulls"] ).transpose().astype( { "Nulls": int } ),
#   pd.DataFrame( [ df.dtypes.to_dict() ], index = [ "Data_Types"] ).transpose().astype( { "Data_Types": 'category' }),
#   pd.DataFrame( [ df.memory_usage(deep=True, index = False).to_dict() ], index = ["Memory"] ).transpose(),
#   df.describe( include = "all" ).transpose().astype( { "count": int } ).rename( columns = { "50%" : "median"} ),
# ], axis = "columns")
# df_info["IRQ"] = df_info["75%"] - df_info["25%"]
# df_info["range"] = df_info["max"] - df_info["min"]
# df_info["sum"] = df_info["mean"] * df_info["count"]
# df_info.insert( 1, "Nulls_pct" , round( df_info["Nulls"] / df_info.shape[0], 2) )

# # Display some info from the meta data
# print( f"Memory: { (df_info['Memory'].sum() / 1000 / 1000 ):_} MB" )
# print( df_info["Data_Types"].value_counts().to_json() )
# df_info.sort_values(by = ["Data_Types", "max"], ascending=[1,0])


### I dropped way more columns becasue some of them don't really help my analysis. Also you have the option to either one-hot encode using (get_dummies) or label encode it if the columns have a natural ordinal flow and that's what i used. These are the columns i kept and that's how i encoded it

In [232]:
object_columns.shape

(2637, 37)

In [233]:
house_object_int2 = house_object_int.copy()
house_object_int.drop(['Lot Config', 'Neighborhood', 'Condition 1',
'Condition 2', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Exter Qual',
'Foundation', 'BsmtFin Type 2', 'Heating', 'Functional', 'Garage Type',
'Garage Qual', 'Sale Type', 'Sale Condition'], axis=1, inplace=True)


KeyError: "['Lot Config', 'Neighborhood', 'Condition 1', 'Condition 2', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Foundation', 'BsmtFin Type 2', 'Heating', 'Functional', 'Garage Type', 'Garage Qual', 'Sale Type', 'Sale Condition'] not found in axis"

In [ ]:
house_object_int.shape

In [ ]:
#each unique value will create a new column for onehot encoding.
#categorical and ordinal use the labelencoder because it doesnt require that
# Create an instance of labelencoder
labelencoder = LabelEncoder()

# Here we encode our categorical variables into integers.
columns_to_encode = ['MS Zoning',
 'Street',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Land Slope',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Exter Cond',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'Heating QC',
 'Central Air',
 'Electrical',
 'Kitchen Qual',
 'Garage Finish',
 'Garage Cond',
 'Paved Drive']
# For loop to integer encode selected categorical columns
for col in columns_to_encode:
    house_object_int[col] = labelencoder.fit_transform(house_object_int[col])

#rememer to drop everything else before this operation, then concatenate these object columns with the rest (integers and floats)

In [ ]:
Combined = pd.concat([numerical_columns, house_object_int], axis=1)

In [ ]:
Combined.isna().sum()

### Display your object columns

## Exploratory Data Analysis

In [ ]:
#plt.figure(figsize=(15, 8))
#sns.boxplot(data=df)
#plt.xticks(rotation=90)  # Rotate x-axis labels for readability
#plt.title("Boxplot of All Features - df")
#plt.savefig('box-whiskers-df.png')
#plt.show()

In [ ]:
#copy data frame with only integer values for a correlation plot
corr = Combined.corr()
plt.figure(figsize=(30,25))
sns.heatmap(corr, annot=True, cmap='RdYlGn') ;
plt.savefig('combined-heat.png')

In [ ]:
plt.figure(figsize=(10,10))
correlation_matrix = Combined.corr()
sns.heatmap(data=correlation_matrix, annot=False, cmap='RdYlBu') ;
plt.title("Correlation Matrix")
plt.show()

Too much detail to visually make sense of it

In [ ]:
#sns.pairplot(df)
#plt.savefig('pairplot.png')

Too much detail to visually make sense of it. So I printed it, crossed out variables that had no strong meaning and highlighted those with strong relationship. The following were identified:

## Data Processing

Regression analysis using Statsmodel

### First clean everything else before processing the data for easier readability

In [ ]:
X = Combined.drop('SalePrice', axis = 1)
y = Combined['SalePrice']

In [ ]:
def rmspe(y_test, y_pred):
  '''
  This function takes y_test and y_pred and calculates the RMSPE'''
  return np.sqrt(np.mean(np.square((y_test - y_pred) / y_test))) * 100

In [208]:
Combined.isna().sum()

,0
PID,0
MS SubClass,0
Lot Frontage,449
Lot Area,0
Overall Qual,0
Overall Cond,0
Year Built,0
Year Remod/Add,0
Mas Vnr Area,23
BsmtFin SF 1,1


In [207]:
# Perform CV
n = 500
results = np.zeros(n)
for idx in range(n):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  results[idx] = rmspe(y_test,y_pred)
print(f"CV RMSPE: {results.mean().round(2)}%")
print(f"Number of Predictors: {len(X.columns)}")

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [22]:
# Fit a linear model using statsmodels
#def fit_linear_model(endog, exog):
  #if endog.dtype == object or exog is not None and exog.dtype == object:
    #  raise ValueError("Pandas data cast to numpy dtype of object. "
   #                   "Check input data with np.asarray(data).")
  #else:
 #     myfit = sm.OLS(y_train, X_train).fit()
   #   myfit.summary()


In [23]:
myfit = sm.OLS(y_train, X_train).fit()
myfit.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

## Data Visualization

## Conclusion

A data dictionary file is available at AWS S3 at [Housing Data Dictionary]( https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Housing%20-%20Data%20Documentation.pdf ).

The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv .
